In [46]:
from product.stock_category import StockCategory
from database.market import Market
from database.sec import SEC
from datetime import datetime, timedelta
from processor.processor import Processor as p
import pytz
import pandas as pd
from tqdm import tqdm
import numpy as np

In [22]:
factors = ['AccumulatedOtherComprehensiveIncomeLossNetOfTax',
 'Assets',
 'AssetsCurrent',
 'CashAndCashEquivalentsAtCarryingValue',
 'EarningsPerShareBasic',
 'EarningsPerShareDiluted',
 'EntityCommonStockSharesOutstanding',
 'Goodwill',
 'IncomeTaxExpenseBenefit',
 'LiabilitiesAndStockholdersEquity',
 'LiabilitiesCurrent',
 'OtherAssetsNoncurrent',
 'PropertyPlantAndEquipmentNet',
 'RetainedEarningsAccumulatedDeficit',
 'filed',
 'StockholdersEquity',
 'NetIncomeLoss']

In [16]:
market = Market()
sec = SEC()

## Currently not modeling stock category due to low actual accuracy

In [6]:
params = {}
product = StockCategory(params)
product.subscribe()
# sim = product.create_sim()
# product.load()
# # product.transformed

In [9]:
market.connect()
sp5 = market.retrieve("sp500")
market.disconnect()

In [38]:
a = ["a","b","c","f"]
b = ["b","c","d","e"]

initial = [x for x in b if x in a]
final = [x for x in a if x in initial]
final

['b', 'c']

In [53]:
sec.connect()
market.connect()
cik = sp5[sp5["Symbol"]==ticker]["CIK"].item()
filing = sec.retrieve_filing_data(cik)
filing["ticker"] = ticker
filing["date"] = [datetime.strptime(str(x),"%Y%m%d") for x in filing["filed"]]
filing = p.column_date_processing(filing)
centralized = list(filing.columns)
for ticker in tqdm(sp5["Symbol"]):
    try:
        cik = sp5[sp5["Symbol"]==ticker]["CIK"].item()
        filing = sec.retrieve_filing_data(cik)
        filing["ticker"] = ticker
        filing["date"] = [datetime.strptime(str(x),"%Y%m%d") for x in filing["filed"]]
        filing = p.column_date_processing(filing)
        centralized = list(set(centralized + list(filing.columns)))
#         if filing.index.size > 40:
#             initial = [x for x in filing.columns if x in centralized]
#             centralized = [x for x in centralized if x in initial]
#         market.store("unified_financials",relevant)
    except Exception as e:
        print(ticker, str(e))
sec.disconnect()
market.disconnect()

 39%|███████████████████▌                              | 198/505 [01:28<02:15,  2.26it/s]

FRC 'filed'


 83%|█████████████████████████████████████████▎        | 417/505 [02:48<00:37,  2.36it/s]

SBNY 'filed'


100%|██████████████████████████████████████████████████| 505/505 [03:15<00:00,  2.58it/s]


In [55]:
centralized.sort()
centralized

['a10yearsinglefamilyresidentialagreement',
 'a199aqualifiedbusinessincome',
 'a2-30seniornotesdue2024',
 'a2-50seniornotesdue2018',
 'a2-625seniornotesdue2020',
 'a2-95seniornotesdue2029',
 'a2007retiredstockrepurchaseprogramnumberofsharesauthorizedtoberepurchased',
 'a2009taxclaimrelatedtonucleardecommissioningliability',
 'a2011lumpsumpaymenttoeachuawemployee',
 'a2011retiredstockrepurchaseprogramauthorizedamount',
 'a2011retiredstockrepurchaseprogramnumberofsharesauthorizedtoberepurchased',
 'a2011sharerepurchaseprogamsharespurchased',
 'a2011sharerepurchaseprogamsharespurchasedin2012',
 'a20131eetcclassbinterestrate',
 'a20131eetcclasscinterestrate',
 'a20131eetcclassinterestrate',
 'a20132bceetcamount',
 'a2013sharerepurchaseprogramcost',
 'a2013sharerepurchaseprogramsharespurchased',
 'a2013sharerepurchaseprogramsharesrepurchased',
 'a2013stockrepurchaseprogramauthorizedrepurchasepriceaspercentofeatonclosingsharepricehighrange',
 'a2013stockrepurchaseprogramauthorizedrepurchasep

In [52]:
maybe = ['assets',
 'liabilitiesandstockholdersequity',
 'netcashprovidedbyusedinfinancingactivities',
 'netcashprovidedbyusedininvestingactivities']

In [ ]:
start = data["year"].min()
end = data["year"].max()
yearly_gap = 1
training_years = product.params["training_years"]
product.db.connect()
the_cats = []
for year in tqdm(range(2012,2021)):
    try:
#         pca_model = PCA(n_components=3)
#         pca_model.fit(test_set)
#         new_set = pca_model.transform(test_set)
#         visuals = pd.DataFrame(new_set,columns=["pca1","pca2","pca3"])
        training_set = data[(data["year"] < year) & (data["year"] > year - training_years)].copy()
        prediction_set = data[(data["year"]==year)].copy()
        non_relevant = ["quarter","ticker","year","training_years"]
        non_relevant.extend([str(x) for x in range(31)])
        factors = [x for x in training_set.columns if x not in non_relevant]
        kmeans = KMeans(n_clusters=10,random_state=42).fit(training_set[factors])
        cats = kmeans.predict(prediction_set[factors])
        prediction_set["cats"] = cats
        the_cats.append(prediction_set[['year','quarter','ticker','cats']])
    except Exception as e:
        print(str(e))
product.db.disconnect()

In [3]:
# ## ai code
# year = 2018
# params = {"training_years":4}
# training_years = 5
# transformed = product.initial_transform()
# prediction_set = product.create_prediction_set(transformed,year,training_years,1)
# training_set = product.create_training_set(transformed,year,training_years,1)